## full_first_model submission

In [8]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import VGG16
import efficientnet
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Sequential, Model # Model : 입력과출력만 넣어주면됌
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import cv2 as cv   # opencv (이미지 resize)

C:\Users\Lenovo\anaconda3\envs\machine_TF2\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [6]:
train_dir = './data/train_full/train'
valid_dir = './data/train_full/validation'

IMAGE_SIZE = 256  # Image 크기 256x256
BATCH_SIZE = 8   # ImageGenerator가 한번에 가져오는 개수
LEARNING_RATE = 5e-5

# ImageDataGenerator를 위한 전처리 함수
# 여기서의 역할은 이미지 resize
def generate_preprocessing(img):
    # 여러가지 작업이 포함될 수 있어요
    img = cv.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img

# ImageDataGenerator부터 만들어요
# train data는 augmentation 해서 생성할 꺼예요!
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=generate_preprocessing,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=generate_preprocessing,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    shuffle=False)


Found 2788 images belonging to 88 classes.
Found 1489 images belonging to 88 classes.


In [9]:
# Pretrained Network
pretrained_network = efn.EfficientNetB4(weights='imagenet',
                                        include_top=False,
                                        input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                        classes=88)

pretrained_network.trainable = False # 학습이 안되게끔 동결처리

'''
model = Sequential()

model.add(pretrained_network)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=88,
                activation='softmax'))
'''

model= Sequential()
model.add(pretrained_network) 
model.add(Flatten()) 

#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=512))

# model.add(Dense(512,activation=('relu'))) 
# model.add(Dense(256,activation=('relu'))) 
# # model.add(Dropout(.3))
# model.add(Dense(128,activation=('relu')))
# # model.add(Dropout(.2))

model.add(Dense(88,activation=('softmax'))) 

#Checking the final model summary
model.summary()

71892992/71892840 [==============================] - 13s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b4 (Functional) (None, 8, 8, 1792)        17673816  
_________________________________________________________________
flatten (Flatten)            (None, 114688)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              117441536 
_________________________________________________________________
dense_1 (Dense)              (None, 88)                90200     
Total params: 135,205,552
Trainable params: 117,531,736
Non-trainable params: 17,673,816
_________________________________________________________________


In [11]:
checkpoint_path = './ckpt/efficientnet_000030-0.845430-0.891367.ckpt'

model.load_weights(checkpoint_path)

OpError: 

In [4]:
labels = ['bottle-broken_large', 'bottle-broken_small', 'bottle-contamination', 'bottle-good', 'cable-bent_wire', 'cable-cable_swap', 'cable-combined', 'cable-cut_inner_insulation', 'cable-cut_outer_insulation', 'cable-good', 'cable-missing_cable', 'cable-missing_wire', 'cable-poke_insulation', 'capsule-crack', 'capsule-faulty_imprint', 'capsule-good', 'capsule-poke', 'capsule-scratch', 'capsule-squeeze', 'carpet-color', 'carpet-cut', 'carpet-good', 'carpet-hole', 'carpet-metal_contamination', 'carpet-thread', 'grid-bent', 'grid-broken', 'grid-glue', 'grid-good', 'grid-metal_contamination', 'grid-thread', 'hazelnut-crack', 'hazelnut-cut', 'hazelnut-good', 'hazelnut-hole', 'hazelnut-print', 'leather-color', 'leather-cut', 'leather-fold', 'leather-glue', 'leather-good', 'leather-poke', 'metal_nut-bent', 'metal_nut-color', 'metal_nut-flip', 'metal_nut-good', 'metal_nut-scratch', 'pill-color', 'pill-combined', 'pill-contamination', 'pill-crack', 'pill-faulty_imprint', 'pill-good', 'pill-pill_type', 'pill-scratch', 'screw-good', 'screw-manipulated_front', 'screw-scratch_head', 'screw-scratch_neck', 'screw-thread_side', 'screw-thread_top', 'tile-crack', 'tile-glue_strip', 'tile-good', 'tile-gray_stroke', 'tile-oil', 'tile-rough', 'toothbrush-defective', 'toothbrush-good', 'transistor-bent_lead', 'transistor-cut_lead', 'transistor-damaged_case', 'transistor-good', 'transistor-misplaced', 'wood-color', 'wood-combined', 'wood-good', 'wood-hole', 'wood-liquid', 'wood-scratch', 'zipper-broken_teeth', 'zipper-combined', 'zipper-fabric_border', 'zipper-fabric_interior', 'zipper-good', 'zipper-rough', 'zipper-split_teeth', 'zipper-squeezed_teeth']
len(labels)

88

In [12]:
from tensorflow.keras.models import load_model

new_model = load_model('./model/ad_ENb4_kaggle.h5')

TypeError: ('Keyword argument not understood:', 'keepdims')

In [3]:
# labels = ['transistor-good', 'capsule-good', 'wood-good', 'bottle-good',
#        'screw-good', 'cable-bent_wire', 'carpet-hole', 'hazelnut-good',
#        'pill-pill_type', 'cable-good', 'metal_nut-scratch', 'pill-good',
#        'screw-thread_side', 'zipper-fabric_border', 'leather-good',
#        'pill-scratch', 'toothbrush-good', 'hazelnut-crack',
#        'screw-manipulated_front', 'zipper-good', 'tile-good',
#        'carpet-good', 'metal_nut-good', 'bottle-contamination',
#        'grid-good', 'zipper-split_teeth', 'pill-crack', 'wood-combined',
#        'pill-color', 'screw-thread_top', 'cable-missing_cable',
#        'capsule-squeeze', 'zipper-rough', 'capsule-crack', 'capsule-poke',
#        'metal_nut-flip', 'carpet-metal_contamination', 'metal_nut-color',
#        'transistor-bent_lead', 'zipper-fabric_interior', 'leather-fold',
#        'tile-glue_strip', 'screw-scratch_neck', 'screw-scratch_head',
#        'hazelnut-cut', 'bottle-broken_large', 'bottle-broken_small',
#        'leather-cut', 'cable-cut_outer_insulation',
#        'zipper-squeezed_teeth', 'toothbrush-defective',
#        'cable-cut_inner_insulation', 'pill-contamination',
#        'cable-missing_wire', 'carpet-thread', 'grid-broken',
#        'pill-faulty_imprint', 'hazelnut-hole', 'leather-glue',
#        'leather-poke', 'transistor-damaged_case', 'wood-scratch',
#        'tile-gray_stroke', 'capsule-faulty_imprint', 'grid-glue',
#        'zipper-combined', 'carpet-color', 'grid-bent', 'pill-combined',
#        'hazelnut-print', 'cable-combined', 'capsule-scratch',
#        'metal_nut-bent', 'zipper-broken_teeth', 'tile-oil',
#        'transistor-misplaced', 'grid-thread', 'grid-metal_contamination',
#        'carpet-cut', 'wood-color', 'cable-cable_swap', 'tile-crack',
#        'leather-color', 'cable-poke_insulation', 'transistor-cut_lead',
#        'wood-hole', 'tile-rough', 'wood-liquid']
# len(labels)

88

In [45]:
from PIL import Image
import numpy as np

image = Image.open('./data/test/20012.png')
image = image.resize((256,256))
image = np.array(image)
x_test = np.array([image])
x_test = (x_test)/255

y_predict = new_model.predict(x_test)

label = labels[y_predict[0].argmax()]
confidence = y_predict[0][y_predict[0].argmax()]
print(label, confidence) #

ValueError: in user code:

    C:\Users\Lenovo\anaconda3\envs\machine_TF2\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\Lenovo\anaconda3\envs\machine_TF2\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Lenovo\anaconda3\envs\machine_TF2\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Lenovo\anaconda3\envs\machine_TF2\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Lenovo\anaconda3\envs\machine_TF2\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\L

In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt

submission_df = pd.read_csv('./data/sample_submission.csv', index_col='index')
test_df = pd.read_csv('./data/test_df.csv', index_col='index')

dir_path = './data/test'

In [4]:
submission_df

,label
index,
0,tile-good
1,tile-good
2,tile-good
3,tile-good
4,tile-good
...,...
2149,tile-good
2150,tile-good
2151,tile-good


In [6]:
from PIL import Image
import numpy as np

i = 0
for num, f_name, label in zip(range(len(test_df)) ,test_df['file_name'], submission_df['label']):
#     print(num, f_name, label)
    try:
        image_path = os.path.join(dir_path, f_name)
        image_path = image_path.replace('\\','/')
        image = Image.open(image_path)
        image = image.resize((256,256))
        image = np.array(image)
        x_test = np.array([image])
        x_test = (x_test)/255
        y_predict = new_model.predict(x_test)
        label = labels[y_predict[0].argmax()]
        submission_df.iloc[num]['label'] = label
        
    except:
        print(num, f_name)
        
display(submission_df)

12 20012.png
23 20023.png
25 20025.png
34 20034.png
48 20048.png
63 20063.png
89 20089.png
94 20094.png
111 20111.png
116 20116.png
120 20120.png
129 20129.png
130 20130.png
144 20144.png
147 20147.png
148 20148.png
150 20150.png
161 20161.png
168 20168.png
177 20177.png
179 20179.png
195 20195.png
196 20196.png
209 20209.png
212 20212.png
221 20221.png
231 20231.png
232 20232.png
234 20234.png
242 20242.png
243 20243.png
267 20267.png
271 20271.png
279 20279.png
286 20286.png
295 20295.png
304 20304.png
307 20307.png
321 20321.png
326 20326.png
338 20338.png
341 20341.png
371 20371.png
385 20385.png
421 20421.png
431 20431.png
447 20447.png
456 20456.png
481 20481.png
493 20493.png
499 20499.png
515 20515.png
525 20525.png
534 20534.png
535 20535.png
556 20556.png
566 20566.png
567 20567.png
569 20569.png
576 20576.png
578 20578.png
579 20579.png
593 20593.png
611 20611.png
617 20617.png
654 20654.png
656 20656.png
665 20665.png
688 20688.png
692 20692.png
766 20766.png
767 20767.png


,label
index,
0,tile-oil
1,grid-good
2,transistor-good
3,metal_nut-good
4,tile-oil
...,...
2149,tile-oil
2150,tile-good
2151,grid-good


**에러**   
ValueError: Input 0 of layer sequential_5 is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: [None, 256, 256]

In [7]:
submission_df.to_csv('./data/submission/submission_0505.csv')